In [1]:
#r "nuget: Microsoft.SemanticKernel, *-*"

Installed Packages Microsoft.SemanticKernel, 0.17.230704.3-preview

In [2]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.SemanticFunctions;
using Microsoft.SemanticKernel.SkillDefinition;
using Microsoft.SemanticKernel.TemplateEngine;
using System.Collections.Generic;
using System.Threading.Tasks;

In [3]:

IKernel kernel = Kernel.Builder.WithAzureTextCompletionService("GPT3Model", "Your AOAI API Endpoint", "Your AOAI API Key").Build();
            

In [4]:
string skPrompt = """

請幫我把 {{$input}} 進行分類，型別包括天氣，課程，產生式，如果不清楚，請回答不確認，參考如下：

問: 會下雨嗎？ 類別:天氣
問: 今天溫度？ 類別:天氣
問: 適度多少？ 類別:天氣
問: 什麼是新能源車？ 類別: 課程
問: 電動車的特點 類別: 課程
問: 概念是什麼？ 類別: 課程
問: 課程相關的內容有哪些？ 類別: 課程
問: 寫一首詩歌？ 類別: 產生式
問: 翻譯一下 類別: 產生式
問: 計算結果 類別: 產生式

天氣輸出 1 ， 課程輸出 2 ， 產生式輸出 3 ，不確認輸出 0，並參考以下格式輸出

{""question"":""{{$input}}"",""label"":""{{$label}}""}

""";

In [5]:
var promptConfig = new PromptTemplateConfig
{
    Completion =
    {
      MaxTokens = 60,
      FrequencyPenalty = (float)0,
      PresencePenalty = (float)0
    }
};

In [6]:
var promptTemplate = new PromptTemplate(
    skPrompt,                        // Prompt template defined in natural language
    promptConfig,                    // Prompt configuration
    kernel                           // SK instance
);

In [7]:
skPrompt


請幫我把 {{$input}} 進行分類，型別包括天氣，課程，產生式，如果不清楚，請回答不確認，參考如下：

問: 會下雨嗎？ 類別:天氣
問: 今天溫度？ 類別:天氣
問: 適度多少？ 類別:天氣
問: 什麼是新能源車？ 類別: 課程
問: 電動車的特點 類別: 課程
問: 概念是什麼？ 類別: 課程
問: 課程相關的內容有哪些？ 類別: 課程
問: 寫一首詩歌？ 類別: 產生式
問: 翻譯一下 類別: 產生式
問: 計算結果 類別: 產生式

天氣輸出 1 ， 課程輸出 2 ， 產生式輸出 3 ，不確認輸出 0，並參考以下格式輸出

{""question"":""{{$input}}"",""label"":""{{$label}}""}


In [8]:

var functionConfig = new SemanticFunctionConfig(promptConfig, promptTemplate);

In [9]:
var classificationFunction = kernel.RegisterSemanticFunction("QASkill", "QuestionClassification", functionConfig);

In [10]:
classificationFunction


{"Name":"QuestionClassification","SkillName":"QASkill","Description":"","IsSemantic":true,"IsSensitive":false,"TrustServiceInstance":{},"RequestSettings":{"Temperature":0,"TopP":0,"PresencePenalty":0,"FrequencyPenalty":0,"MaxTokens":60,"StopSequences":[],"ResultsPerPrompt":1,"ChatSystemPrompt":"Assi... Name QuestionClassification SkillName QASkill Description IsSemantic True IsSensitive False TrustServiceInstance Microsoft.SemanticKernel.Security.TrustService RequestSettings Microsoft.SemanticKernel.AI.TextCompletion.CompleteRequestSettings Temperature 0 TopP 0 PresencePenalty 0 FrequencyPenalty 0 MaxTokens 60 StopSequences ResultsPerPrompt 1 ChatSystemPrompt Assistant is a large language model. TokenSelectionBiases (empty) Parameters index value 0 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name input Description DefaultValue 1 Microsoft.SemanticKernel.SkillDefinition.ParameterView Name label Description DefaultValue

In [11]:
var input = "今天廣州天氣怎麼樣？";

In [12]:
var classification = await kernel.RunAsync(input, classificationFunction);

Console.WriteLine(classification);


答案：{"question":"今天廣州天氣怎麼樣？","label":"1"}
